In [1]:
import numpy as np

%load_ext autoreload
%autoreload 2
import my_baselines

# Purchase prediction

In [2]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]
data[0]

{'categories': [['Clothing, Shoes & Jewelry', 'Women'],
  ['Clothing, Shoes & Jewelry',
   'Novelty, Costumes & More',
   'Novelty',
   'Clothing',
   'Women',
   'Leggings']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I402344648',
 'rating': 4.0,
 'reviewHash': 'R798569390',
 'reviewText': "The model in this picture has them rolled up at the top because they are actually very high waisted! that's my only complaint though, because they are very good quality, and fit really well! I am 5'2&#34; 120lbs with thick thighs and i love them i can't wait to wear them out!",
 'reviewTime': '09 26, 2013',
 'reviewerID': 'U490934656',
 'summary': 'High Waisted',
 'unixReviewTime': 1380153600}

In [3]:
##########################
# 2
##########################

trainX, valX = data[:100000], data[100000:]

reviewer_item_pair = {}
reviewer_list = []
item_list = []

for d in data:
    reviewer = d['reviewerID']
    item = d['itemID']
    pair = reviewer_item_pair.get(reviewer, [])
    pair.append(item)
    reviewer_item_pair[reviewer] = pair
    reviewer_list.append(reviewer)
    item_list.append(item)
    
import random
cnt = 0
neg_pair = []

while(cnt < 100000):
    reviewer = random.choice(reviewer_list)
    item = random.choice(item_list)
    if item not in reviewer_item_pair[reviewer]:
        neg_pair.append((reviewer, item))
        cnt += 1    
        
new_valX = []
new_valY = []

for d in valX:
    new_valX.append((d['reviewerID'], d['itemID']))
    new_valY.append(1)

new_valX += neg_pair
new_valY += len(neg_pair) * [0]

In [4]:
##########################
# 2
##########################
item_popularity = {}

for d in data:
    item = d['itemID']
    p = item_popularity.get(item, 0)
    item_popularity[item] = p + 1

item_popularity_rank = sorted(item_popularity.items(), key=lambda x: x[1], reverse=True)
item_rank = {x[0]:rank for rank, x in enumerate(item_popularity_rank)}

def p2(u, i):
    try:
        rank = item_rank[i]
    except KeyError:
        return 0
    if rank < len(item_rank):
        return 1
    else:
        return 0

my_baselines.purchaseBaseline(p2)  

In [5]:
##########################
# 3
##########################
reviewer_cat_pair = {}
item_cat_pair = {}

for d in data:
    reviewer = d['reviewerID']
    item = d['itemID']
    cat = d['categories']
    
    reviewer_pair = reviewer_cat_pair.get(reviewer, [])
    item_pair = item_cat_pair.get(item, [])
    for c in cat:
        if c not in reviewer_pair:
            reviewer_pair.append(c)
        if c not in item_pair:
            item_pair.append(c)
        
    reviewer_cat_pair[reviewer] = reviewer_pair
    item_cat_pair[item] = item_pair
    
def p3(u, i):
    try:
        reviewer_pair = reviewer_cat_pair[u]
        item_pair = item_cat_pair[i]
    except KeyError:
        return 0
    
    for i in item_pair:
        if i in reviewer_pair: return 1
    return 0

my_baselines.purchaseBaseline(p3)

In [6]:
##########################
# 4
##########################
kaggle_username = 'Fanjin'
kaggle_username

'fanjin'

# Rating prediction

In [13]:
##########################
# 5
##########################
alpha = 0.
for d in trainX:
    alpha += d['rating']
alpha /= len(trainX)


def p5(u, i):
    return alpha

# my_baselines.ratingBaseline(p5)

In [18]:
##########################
# 6
##########################

beta_user = {}
beta_item = {}

for d in trainX:
    reviewer = d['reviewerID']
    item = d['itemID']
    rating = d['rating']
    
    bu = beta_user.get(reviewer, (0., 0))
    bi = beta_item.get(item, (0., 0))
    
    bu = (bu[0]+rating, bu[1]+1)
    bi = (bi[0]+rating, bi[1]+1)
    
    beta_user[reviewer] = bu
    beta_item[item] = bi
    
beta_user = {r:(b[0]/b[1] - alpha) for r,b in beta_user.items()}
beta_item = {r:(b[0]/b[1] - alpha) for r,b in beta_item.items()}

def p6(u, i):
    bu = beta_user.get(u, 0.)
    bi = beta_item.get(i, 0.)
    
    return alpha + bu + bi

# my_baselines.ratingBaseline(p6)

In [27]:
##########################
# 7
##########################

beta_user_rank = sorted(beta_user.items(), key=lambda x: x[1])
beta_item_rank = sorted(beta_item.items(), key=lambda x: x[1])

print('Largest beta')
print('User# ' + beta_user_rank[-1][0])
print('Item# ' + beta_item_rank[-1][0])

print('Smallest beta')
print('User# ' + beta_user_rank[0][0])
print('Item# ' + beta_item_rank[0][0])

Largest beta
User# U529851781
Item# I917400017
Smallest beta
User# U757790423
Item# I784810620


In [10]:
##########################
# 8
##########################

lmbda = [0.01, 0.1, 1, 2, 5, 10]